In [7]:
import os
import io
import cv2
import tqdm
import torch

#from torchvision import transforms
from torchvision import models, transforms
import matplotlib.pyplot as plt
from pillow_heif import register_heif_opener
from PIL import Image
from ultralytics import YOLO

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from googleapiclient.http import MediaFileUpload, MediaIoBaseDownload

In [8]:
register_heif_opener()

# -------- 1. Google Drive Authentication --------
SCOPES = ["https://www.googleapis.com/auth/drive"]
creds = None

if os.path.exists("token.json"):
    creds = Credentials.from_authorized_user_file("token.json", SCOPES)
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            "client_secret_676204196755-36uo6inh0emqfied3g2s0185anhbe1l5.apps.googleusercontent.com.json", SCOPES)
        creds = flow.run_local_server(port=0)
with open("token.json", "w") as token:
    token.write(creds.to_json())

try:
    # -------- 2. Google Drive "same cat/same dog", Downloading images/videos --------
    service = build("drive", "v3", credentials=creds)
    response = service.files().list(
        q="sharedWithMe and mimeType='application/vnd.google-apps.folder' and name contains 'Capstone'",
        spaces="drive"
    ).execute()
    capstone_folders = response.get('files', [])

    if not capstone_folders:
        print('No Capstone folder found.')
        exit()

    # Looking inside "same cat", "same dog"
    children_folders = []
    for parent in capstone_folders:
        parent_id = parent['id']
        response = service.files().list(
            q=f"'{parent_id}' in parents and mimeType='application/vnd.google-apps.folder'",
            spaces="drive"
        ).execute()
        for child in response.get('files', []):
            if child['name'] in ['same cat', 'same dog']:
                children_folders.append(child)

    if not children_folders:
        print('No same cat or same dog folder found.')
        exit()

    for fold in children_folders:
        fold_name = fold['name']
        fold_id = fold['id']

        # 2-1. Saving in Local Directory
        save_img_dir = os.path.join('downloaded_data', fold_name)
        os.makedirs(save_img_dir, exist_ok=True)

        # 2-2. Downloading Images/Videos
        response = service.files().list(
            q=f"'{fold_id}' in parents and (mimeType contains 'image/' or mimeType contains 'video/')",
            spaces='drive',
            fields="files(id,name,mimeType)",
            pageSize=1000,
        ).execute()
        files = response.get('files', [])

        for file in tqdm.tqdm(files, desc=f"[{fold_name}] Download"):
            file_id = file['id']
            file_name = file['name']
            mime_type = file['mimeType']
            local_path = os.path.join(save_img_dir, file_name)

            if os.path.exists(local_path):
                print(f"Skipping download, already exists: {local_path}")
            else:
                request = service.files().get_media(fileId=file_id)
                fh = io.BytesIO()
                downloader = MediaIoBaseDownload(fh, request)
                done = False
                while not done:
                    status, done = downloader.next_chunk()
                fh.seek(0)
                with open(local_path, "wb") as out_f:
                    out_f.write(fh.getvalue())

        # 2-3. Extracting images every 0.25 seconds from videos
        video_exts = ('.mov', '.mp4', '.avi', '.mkv', '.wmv', '.flv', '.webm')
        for fname in os.listdir(save_img_dir):
            if fname.lower().endswith(video_exts):
                base_name = os.path.splitext(fname)[0]
                example_frame = os.path.join(save_img_dir, f"{base_name}_frame000000.jpg")
                if os.path.exists(example_frame):
                    print(f"Skipping {fname}: frames already extracted")
                    continue
                video_path = os.path.join(save_img_dir, fname)
                cap = cv2.VideoCapture(video_path)
                fps = cap.get(cv2.CAP_PROP_FPS)
                interval = int(fps * 0.25) if fps > 0 else 1
                frame_id = 0
                save_id = 0
                while cap.isOpened():
                    ret, frame = cap.read()
                    if not ret:
                        break
                    if frame_id % max(1, interval) == 0:
                        out_name = f"{base_name}_frame{save_id:06d}.jpg"
                        out_path = os.path.join(save_img_dir, out_name)
                        cv2.imwrite(out_path, frame)
                        save_id += 1
                    frame_id += 1
                cap.release()
                print(f"{fname}: {save_id} frames extracted")

except HttpError as e:
    print("Error: ", str(e))

print("Google Drive same cat/dog Download and Frame extraction complete!")

# -------- 3. Standardization + Augmentation (For all images) --------
print("Augmentation/Standardization start...")

# Standardization + Augmentation transform
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

folders = ['downloaded_data/same cat', 'downloaded_data/same dog']
out_base = 'augmented_data'
os.makedirs(out_base, exist_ok=True)
img_exts = ('.jpg', '.jpeg', '.png', '.webp', '.heic')

for folder in folders:
    class_name = os.path.basename(folder)
    out_folder = os.path.join(out_base, class_name)
    os.makedirs(out_folder, exist_ok=True)

    img_files = [f for f in os.listdir(folder) if f.lower().endswith(img_exts)]
    for fname in img_files:
        img_path = os.path.join(folder, fname)
        try:
            img = Image.open(img_path).convert('RGB')
        except Exception as e:
            print(f"Fail open {img_path}: {e}")
            continue

        # Saving pictures including the original ones
        orig_fname = f"{os.path.splitext(fname)[0]}_orig.jpg"
        orig_path = os.path.join(out_folder, orig_fname)
        img.resize((224,224)).save(orig_path)
        # Original picture + 5 randomly standardized/augmented pictures
        for aug_id in range(5):
            img_aug_tensor = transform(img)
            aug_fname = f"{os.path.splitext(fname)[0]}_aug{aug_id:02d}.jpg"
            aug_path = os.path.join(out_folder, aug_fname)
            to_img = transforms.ToPILImage()(torch.clamp(img_aug_tensor, 0, 1))
            to_img.save(aug_path)

print("All augmented/standardized images saved!")

[same dog] Download: 0it [00:00, ?it/s]
[same cat] Download: 100%|██████████| 137/137 [00:00<00:00, 6285.77it/s]


Skipping download, already exists: downloaded_data\same cat\IMG_9806.HEIC
Skipping download, already exists: downloaded_data\same cat\IMG_9809.HEIC
Skipping download, already exists: downloaded_data\same cat\IMG_9810.HEIC
Skipping download, already exists: downloaded_data\same cat\IMG_9808.HEIC
Skipping download, already exists: downloaded_data\same cat\IMG_9807.HEIC
Skipping download, already exists: downloaded_data\same cat\IMG_9800.HEIC
Skipping download, already exists: downloaded_data\same cat\IMG_9802.HEIC
Skipping download, already exists: downloaded_data\same cat\IMG_9801.HEIC
Skipping download, already exists: downloaded_data\same cat\IMG_9796.HEIC
Skipping download, already exists: downloaded_data\same cat\IMG_9797.HEIC
Skipping download, already exists: downloaded_data\same cat\IMG_9799.HEIC
Skipping download, already exists: downloaded_data\same cat\IMG_9798.HEIC
Skipping download, already exists: downloaded_data\same cat\IMG_9805.HEIC
Skipping download, already exists: dow

In [10]:
# YOLOv8n: Already trained with cat & dog (Index 15:cat, Index 16:dog)
# https://docs.ultralytics.com/datasets/detect/coco/#coco-pretrained-models
model = YOLO('yolov8n.pt')

names_dict = model.model.names
cat_id = [k for k, v in names_dict.items() if v == 'cat'][0]
dog_id = [k for k, v in names_dict.items() if v == 'dog'][0]
person_id = [k for k, v in names_dict.items() if v == 'person'][0]
bear_id = [k for k, v in names_dict.items() if v == 'bear'][0]

# Opening webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    # Utilizing YOLO model detecting (confidence level = 30%)
    results = model.predict(frame, conf=0.3)

    found_pet = False
    reject_flag = False

    for box in results[0].boxes:
        cls = int(box.cls[0])
        label = model.model.names[cls]
        x1, y1, x2, y2 = map(int, box.xyxy[0])

        # Color
        if label == 'cat':
            color = (0, 255, 0)    # Green
            found_pet = True
        elif label == 'dog':
            color = (255, 0, 0)    # Blue
            found_pet = True
        elif label == 'person':
            color = (0, 0, 255)    # Red
            reject_flag = True
        elif label == 'bear':
            color = (0, 0, 255)  # Red
            reject_flag = True
        else:
            color = (0, 255, 255)    # Yellow

        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
        cv2.putText(frame, label, (x1, y1+30), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)

    # Decision Logics
    if reject_flag:
        status = "REJECT (person or bear in screen!)"
    elif found_pet:
        status = "STAGE 2 (only pet present)"
        # stage2(frame)

        '''
        We can have our Stage 2 logics here
        '''
        
    else:
        status = "EMPTY or only allowed objects in screen"

    # Status on Screen
    cv2.putText(frame, status, (30, 40), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0,0,0), 3)
    cv2.putText(frame, status, (30, 40), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0,255,255), 2)

    cv2.imshow('Pet Gate', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


0: 480x640 1 chair, 1 book, 200.6ms
Speed: 6.3ms preprocess, 200.6ms inference, 1.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 82.0ms
Speed: 1.4ms preprocess, 82.0ms inference, 0.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 2 chairs, 78.7ms
Speed: 1.2ms preprocess, 78.7ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 1 refrigerator, 86.6ms
Speed: 1.4ms preprocess, 86.6ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 77.6ms
Speed: 1.5ms preprocess, 77.6ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 1 chair, 86.4ms
Speed: 1.2ms preprocess, 86.4ms inference, 0.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 1 chair, 1 refrigerator, 82.0ms
Speed: 1.8ms preprocess, 82.0ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 1 chair, 129.6ms
Speed: 1.6

In [ ]:
# Checking types of images/videos in folders
#def list_file_types(folder):
#    file_count = {}
#    for fname in os.listdir(folder):
#        ext = os.path.splitext(fname)[-1].lower()
#        file_count[ext] = file_count.get(ext, 0) + 1
#    return file_count

#print("same cat:", list_file_types('downloaded_data/same cat'))
#print("random dogs:", list_file_types('downloaded_data/random dogs'))
#print("random cats:", list_file_types('downloaded_data/random cats'))

same cat: {'.heic': 134, '.mov': 2, '.jpg': 88}
random dogs: {'.jpg': 1000}
random cats: {'.png': 104, '.webp': 1, '.jpg': 3}
